<a href="https://colab.research.google.com/github/KKSTB/COMM061/blob/main/lab06_NER_using_LM_solution_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition using Language Models

This lab will focus on helping you fine-tune Transformers models for Named Entity Recognition. You will use the Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

In [ ]:
from IPython.display import HTML, display
colab_button = HTML(
    '<a target="_blank" href="https://colab.research.google.com/github/surrey-nlp/NLP-2025/blob/main/lab06/lab06-NER_using_LM_solution.ipynb">'
    '<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'
)
display(colab_button)

## Installation
Let us first install the required libraries

In [ ]:
!pip install datasets transformers seqeval huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=3f7f5bdfbe1344239c18ee2ee063f009b02589ebf1a54cadd7ac7bf82e625c92
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolv

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.13
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Need to get 8,489 kB of archives.
After this operation, 7,671 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.6.1 [8,489 kB]
Fetched 8,489 kB in 1s (13.3 MB/s)
debconf: unable to initialize frontend: Dialog
debco

## Dataset(s)

We will use the Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be done with the functions load_dataset and load_metric.

The notebook should work with any token classification dataset provided by the Datasets library.

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, concatenate_datasets
from evaluate import load
datasets = load_dataset("surrey-nlp/PLOD-CW-25")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/268 [00:00<?, ?B/s]

PLOD-CW-25-Train.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

PLOD-CW-25-Test.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

PLOD-CW-25-Val.parquet:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/150 [00:00<?, ? examples/s]

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
# datasets["train_en"] refers to the subset of the Wikineural dataset that contains English articles for training. The code datasets["train_en"][0]
# retrieves the first article in this subset.

datasets["train"][0]

{'tokens': ['MRR',
  ',',
  'mortality',
  'rate',
  'ratio',
  ';',
  'TBI',
  ',',
  'traumatic',
  'brain',
  'injury',
  '.'],
 'pos_tags': ['NOUN',
  'PUNCT',
  'NOUN',
  'NOUN',
  'NOUN',
  'PUNCT',
  'PROPN',
  'PUNCT',
  'ADJ',
  'NOUN',
  'NOUN',
  'PUNCT'],
 'ner_tags': ['B-AC',
  'O',
  'B-LF',
  'I-LF',
  'I-LF',
  'O',
  'B-AC',
  'O',
  'B-LF',
  'I-LF',
  'I-LF',
  'O']}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [ ]:
# The first line defines a list of NER label strings,
# where each label represents a different named entity category,
# such as a person (PER), organization (ORG), location (LOC), or miscellaneous (MISC).

# B (Beginning) indicates the first token of a named entity.
# I (Inside) indicates any non-first token of a named entity.
# O (Outside) indicates that a token is not part of a named entity.

label_list = ['O', 'B-AC', 'B-LF', 'I-LF']

#creates a dictionary called labels_vocab where each label string is a key and the corresponding
#value is an integer that represents that label.

labels_vocab = {'O': 0, 'B-AC': 1, 'B-LF': 2, 'I-LF': 3}

#labels_vocab_reverse dictionary is used to map the integer labels back to their string representations.
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

In [ ]:
# prompt: convert 'ner_tags' in datasets from strings to corresponding integers listed in labels_vocab

def convert_ner_tags_to_int(example):
  """Converts NER tags from strings to integers."""
  example['ner_tags'] = [labels_vocab[tag] for tag in example['ner_tags']]
  return example

# Apply the conversion function to the dataset
datasets = datasets.map(convert_ner_tags_to_int)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
datasets["train"][0]

{'tokens': ['MRR',
  ',',
  'mortality',
  'rate',
  'ratio',
  ';',
  'TBI',
  ',',
  'traumatic',
  'brain',
  'injury',
  '.'],
 'pos_tags': ['NOUN',
  'PUNCT',
  'NOUN',
  'NOUN',
  'NOUN',
  'PUNCT',
  'PROPN',
  'PUNCT',
  'ADJ',
  'NOUN',
  'NOUN',
  'PUNCT'],
 'ner_tags': [1, 0, 2, 3, 3, 0, 1, 0, 2, 3, 3, 0]}

If you want to train a monolingual model (NER on a single language), you have to use a single language for each split.

In [ ]:
train_dataset = datasets["train"]
val_dataset = datasets["validation"]
test_dataset = datasets["test"]

This is how you can concatenate multiple dataset splits to train a Multilingual NER model. However, let us focus on Monolingual NER for now. You can uncomment and use the following for other languages

In [ ]:
# train_dataset = concatenate_datasets([datasets["train_de"],
#                                       datasets["train_en"],
#                                       datasets["train_es"],
#                                       datasets["train_fr"],
#                                       datasets["train_it"],
#                                       datasets["train_nl"],
#                                       datasets["train_pl"],
#                                       datasets["train_pt"],
#                                       datasets["train_ru"]])

# val_dataset = concatenate_datasets([datasets["val_de"],
#                                     datasets["val_en"],
#                                     datasets["val_es"],
#                                     datasets["val_fr"],
#                                     datasets["val_it"],
#                                     datasets["val_nl"],
#                                     datasets["val_pl"],
#                                     datasets["val_pt"],
#                                     datasets["val_ru"]])

# test_dataset = concatenate_datasets([datasets["test_de"],
#                                     datasets["test_en"],
#                                     datasets["test_es"],
#                                     datasets["test_fr"],
#                                     datasets["test_it"],
#                                     datasets["test_nl"],
#                                     datasets["test_pl"],
#                                     datasets["test_pt"],
#                                     datasets["test_ru"]])

## Pre-processing

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

 - we get a tokenizer that corresponds to the model architecture we want to use,
 - we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

#the tokenizer below is the checkpoint
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") ## We use the BERT tokenizer

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can directly call this tokenizer on one sentence, or you can use the tokenize method to tokenize multiple sentences at once.

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In the above output,

token_type_ids,

is used to distinguish between two sequences that are concatenated together. For example, if we have two sentences "A cat is walking" and "The dog is barking", and we concatenate them as input to a model, then the tokenizer will assign a different token_type_id to the tokens in the first sentence (e.g. 0) and to the tokens in the second sentence (e.g. 1). In the output of the tokenizer, each token is assigned a token_type_id which indicates which sequence it belongs to.

attention_mask,

is used to tell the model which tokens to attend to and which ones to ignore. It is a binary vector that has the same length as the input sequence, where each element is either 0 or 1. The 1s indicate the positions of the tokens that should be attended to by the model, and the 0s indicate the positions of the padding tokens that should be ignored.


This is a good overal tutorial for preprocessing for Transformer. Check this out to learn more about the preprocessing steps.
https://huggingface.co/transformers/preprocessing.html

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Just change the value of the following flag:

In [ ]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens(i.e.[CLS],[SEP]) have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = "bert-base-multilingual-cased" ## You Change this to any generic language model from HuggingFace which supports fine-tuning.
args = TrainingArguments(                   ## This is where all your hyperparameters are, reduce *_batch_size if you are working with a GPU which has limited memory.
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16, # you can change the batch size according to the computational resources
    num_train_epochs=1,   ## may not be enough for a large data, increase this to 5 later in your experiments.
    weight_decay=0.01,
    push_to_hub=False,
    eval_steps=10000,
    save_steps=10000,
    report_to="none",

    #learning_rate is a hyperparameter that controls how quickly the model adapts to the training data. It specifies the size of the step taken at each iteration of the training process in the direction of minimizing the loss. A smaller learning rate results in slower but more precise convergence, while a larger learning rate can lead to faster convergence, but may result in overshooting the optimal solution.
    #per_device_train_batch_size :This parameter specifies how many training examples should be processed together in one forward/backward pass.
    #weight_decay: A regularization term added to the loss function to prevent overfitting. It controls the amount of regularization applied to the weights of the model during training.
    #push_to_hub: A flag that specifies whether to push the trained model to the HuggingFace model hub after training. The model hub is a repository of pre-trained models that can be easily shared and used by the community.
    #eval_steps: The number of training steps between each evaluation of the model on the validation set. This parameter is used to speed up the training process by evaluating the model less frequently.
    #num_train_epochs: The number of times to iterate over the entire training set during training. One epoch means one pass over the entire dataset. Increasing this value may improve the performance of the model, but also increases the training time.



)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.seqeval is a Python library for sequence labeling evaluation. It provides metrics such as precision, recall, and F1-score for named entity recognition (NER) and other sequence labeling tasks.

In [ ]:
metric = load("seqeval")

So we will need to do a bit of post-processing on our predictions:

 - select the predicted index (with the maximum logit) for each token
 - convert it to its string label
 - ignore everywhere we set a label of -100

The following function does all this post-processing on the result of Trainer.evaluate (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Before you go any further:
1. Login to the HuggigFace CLI on your system using command ``huggingface-cli login``
2. Get your access token from your user account on HuggingFace at https://huggingface.co/settings/tokens (preferably a write access token)
3. Use this token with the command above to ensure being logged in to HuggingFace.

IF you are not working on your own machine with a GPU and have opened this notebook in Colab / Studio Lab, you can use the also cell below to login. You could have used this cell instead of commandline (cli) login, but where is the fun in that?

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the Trainer:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-24-25887df61a95>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=63, training_loss=0.45616016690693206, metrics={'train_runtime': 70.8642, 'train_samples_per_second': 28.223, 'train_steps_per_second': 0.889, 'total_flos': 215818692353280.0, 'train_loss': 0.45616016690693206, 'epoch': 1.0})

## Evaluation

The evaluate method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.252494215965271,
 'eval_precision': 0.7014925373134329,
 'eval_recall': 0.8084440969507428,
 'eval_f1': 0.7511805303305487,
 'eval_accuracy': 0.9105976406356634,
 'eval_runtime': 2.5239,
 'eval_samples_per_second': 99.055,
 'eval_steps_per_second': 6.339,
 'epoch': 1.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the predict method:

In [ ]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'AC': {'precision': np.float64(0.8125),
  'recall': np.float64(0.8808030112923463),
  'f1': np.float64(0.8452739313666465),
  'number': np.int64(797)},
 'LF': {'precision': np.float64(0.5442622950819672),
  'recall': np.float64(0.6887966804979253),
  'f1': np.float64(0.608058608058608),
  'number': np.int64(482)},
 'overall_precision': np.float64(0.7014925373134329),
 'overall_recall': np.float64(0.8084440969507428),
 'overall_f1': np.float64(0.7511805303305487),
 'overall_accuracy': 0.9105976406356634}

Now, can you try to evaluate the NER task performance on a multilingual dataset? with a different language model?
1. Think about what Language Model on HuggingFace can help you Fine-tune over a multilingual dataset. Hint: XLM-R
2. Go back to the commented out multilingual dataset concatetation notebook cell and try to include a few more languages in the dataset.
3. Try to fine-tune the model on this dataset.
4. Report the performance in a MARKDOWN CELL below, showing your work on multilingual model fine-tuning in the cells below.